# Breaking Wordle Game
目標是破解一款名叫 ： “Ｗordle” 的遊戲 <br>

* 遊戲目標 ： 猜到一個 5 個字母的單字 <br>

* 遊戲流程 ：  <br>
先隨便猜一個字 <br>
接著會根據猜測的字給出提示 <br>
然後接續繼續猜測 <br>
直到猜到正確單字為止 <br>

# import package

In [1]:
# import package
import string
from collections import Counter
from itertools import chain

# load data

In [2]:
# load data
'''
pos : can be fill in the blank
real : may be the real answer
'''
possible_word_path = 'possibility_words.txt'
real_word_path = 'real_words.txt'
pos_file = open(possible_word_path, 'r', encoding='utf-8')
real_file = open(real_word_path, 'r', encoding='utf-8')

# load & combine words

In [3]:
# load words to summarize all letters frequence 
pos_words = [pos_fline.strip().lower() for pos_fline in pos_file if pos_fline.strip()]
real_words = [real_line.strip().lower() for real_line in real_file if real_line.strip()]
comb_words = pos_words + real_words
print(f"loaded pos words : {len(pos_words)}")
print(f"loaded real words : {len(real_words)}")
print(f"loaded combined words : {len(comb_words)}")

loaded pos words : 10657
loaded real words : 2315
loaded combined words : 12972


In [4]:
# filter letters
pos_letters = [pos_letter for pos_word in pos_words for pos_letter in pos_word]
real_letters = [real_letter for real_word in real_words for real_letter in real_word]
comb_letters = [comb_letter for comb_word in comb_words for comb_letter in comb_word]

# count freq
pos_letter_freq = Counter(pos_letters)
real_letter_freq = Counter(real_letters)
comb_letter_freq = Counter(comb_letters)


# sort
sorted_pos_freq = dict(sorted(pos_letter_freq.items(), key=lambda x: x[1], reverse=True))
sorted_real_freq = dict(sorted(real_letter_freq.items(), key=lambda x: x[1], reverse=True))
sorted_comb_freq = dict(sorted(comb_letter_freq.items(), key=lambda x: x[1], reverse=True))

# show result of real word
for letter, freq in sorted_comb_freq.items():
    print(f"{letter}: {freq}")


s: 6665
e: 6662
a: 5990
o: 4438
r: 4158
i: 3759
l: 3371
t: 3295
n: 2952
u: 2511
d: 2453
y: 2074
c: 2028
p: 2019
m: 1976
h: 1760
g: 1644
b: 1627
k: 1505
f: 1115
w: 1039
v: 694
z: 434
j: 291
x: 288
q: 112


# find 1st word

In [5]:
# def func to score word
def score_word(word, freq_dict):
    return sum(freq_dict.get(letter, 0) for letter in set(word))

In [6]:
# run all words to score and find best word
scored_words = [(word, score_word(word, sorted_comb_freq)) for word in comb_words]

best_word = max(scored_words, key=lambda x: x[1])

print(f"best word : {best_word[0]},（score : {best_word[1]}）")


best word : aeros,（score : 27913）


# find second word

接著我們希望找出第 2 個單字 <br>
會區分出是否有猜對的 <br>
出現以下幾種情況 : <br>
1. black : 這個字不存在 <br>
2. yellow : 這個字存在，但位置錯誤 <br>
3. green : 這個字存在，並且位置正確 <br>

應該要製作一個函式 <br>
讓使用者輸入(目前猜的字，顏色的種類(bybbg類似這樣的表現方法)) <br>
最後返回下一個應該猜測的字 <br>

In [7]:
# init candidate words
candidate_words = [word for word in comb_words if len(word) == 5]

# get letter freq
letter_freq = Counter(chain.from_iterable(candidate_words))

# remember history
history = []  # [(guess1, feedback1), (guess2, feedback2), ...]


In [8]:
# def func to filter word
def filter_candidates(history, candidate_words):
    for guess, feedback in history:
        new_candidates = []

        for word in candidate_words:
            match = True
            # mark which letter is match
            used = [False]*5

            # first : solve green
            for i in range(5):
                if feedback[i] == 'g':
                    if word[i] != guess[i]:
                        match = False
                        break
                    # letter place is right
                    used[i] = True

            if not match:
                continue

            # second : solve yellow
            for i in range(5):
                if feedback[i] == 'y':
                    found = False
                    for j in range(5):
                        # place is another, but need to show in word
                        if not used[j] and word[j] == guess[i] and j != i:
                            used[j] = True
                            found = True
                            break
                    if not found:
                        match = False
                        break

            if not match:
                continue

            # third : solve black
            for i in range(5):
                if feedback[i] == 'b':
                    ch = guess[i]
                    allowed = sum(
                        1 for j in range(5) if feedback[j] in ['g', 'y'] and guess[j] == ch
                    )
                    actual = word.count(ch)
                    if actual > allowed:
                        match = False
                        break

            if match:
                new_candidates.append(word)
        # get new maybe word
        candidate_words = new_candidates

    return candidate_words


In [9]:
def next_best_guess(candidate_words, letter_freq):
    if not candidate_words:
        return None
    # score all possible word and return best
    scored_words = [(word, score_word(word, letter_freq)) for word in candidate_words]
    return max(scored_words, key=lambda x: x[1])[0]


In [10]:
def update_history_and_guess(prev_guess, feedback, history, candidate_words, letter_freq):
    # find the answer
    if feedback == "ggggg":
        print("congratulations, you find the answer🎉")
    # update history
    history.append((prev_guess, feedback))
    # update candidate
    filtered_words = filter_candidates(history, candidate_words)
    # get next best guess
    next_guess = next_best_guess(filtered_words, letter_freq)

    return next_guess, filtered_words


# 實際操作 

In [11]:
# init
candidate_words = [word for word in comb_words if len(word) == 5]
letter_freq = Counter(chain.from_iterable(candidate_words))
history = []

In [ ]:
# # init
# candidate_words = [word for word in pos_words if len(word) == 5]
# letter_freq = Counter(chain.from_iterable(candidate_words))
# history = []

In [12]:
# first guess
first_guess = next_best_guess(candidate_words, letter_freq)
print(f"first recommend : {first_guess}")
# get feedback
first_feedback = input(f"enter feedback of '{first_guess}' (like : bybbg): ")


first recommend : aeros


In [13]:
# update & second guess
second_guess, candidate_words = update_history_and_guess(
    first_guess, first_feedback, history, candidate_words, letter_freq
)
print(f"second recommend : {second_guess}")
# get feedback
second_feedback = input(f"enter feedback of '{second_guess}' (like : bybbg): ")


second recommend : lotes


In [45]:
# # update & second guess
# second_guess, candidate_words = update_history_and_guess(
#     "alert", "bybyb", history, candidate_words, letter_freq
# )
# print(f"second recommend : {second_guess}")
# # get feedback
# second_feedback = input(f"enter feedback of '{second_guess}' (like : bybbg): ")


second recommend : loris


In [40]:
# update & third guess
third_guess, candidate_words = update_history_and_guess(
    second_guess, second_feedback, history, candidate_words, letter_freq
)
print(f"second recommend : {third_guess}")
# get feedback
third_feedback = input(f"enter feedback of '{third_guess}' (like : bybbg): ")


second recommend : None


In [ ]:
# # update & third guess
# third_guess, candidate_words = update_history_and_guess(
#     "broil", "byygy", history, candidate_words, letter_freq
# )
# print(f"second recommend : {third_guess}")
# # get feedback
# third_feedback = input(f"enter feedback of '{third_guess}' (like : bybbg): ")


second recommend : loris


In [27]:
# update & fourth guess
fourth_guess, candidate_words = update_history_and_guess(
    third_guess, third_feedback, history, candidate_words, letter_freq
)
print(f"second recommend : {fourth_guess}")
# get feedback
fourth_feedback = input(f"enter feedback of '{fourth_guess}' (like : bybbg): ")


second recommend : loamy


In [28]:
# update & fifth guess
fifth_guess, candidate_words = update_history_and_guess(
    fourth_guess, fourth_feedback, history, candidate_words, letter_freq
)
print(f"second recommend : {fifth_guess}")
# get feedback
fifth_feedback = input(f"enter feedback of '{fifth_guess}' (like : bybbg): ")


second recommend : modal


In [ ]:
# update & sixth guess
sixth_guess, candidate_words = update_history_and_guess(
    fifth_guess, fifth_feedback, history, candidate_words, letter_freq
)
print(f"second recommend : {sixth_guess}")
# get feedback
sixth_feedback = input(f"enter feedback of '{sixth_guess}' (like : bybbg): ")
